In [62]:
import pandas as pd
import csv
import scipy.sparse as sp

In [2]:
sparse_map = pd.read_csv('process_ukb825.csv',header=None)

## Reconstruct rows

In [47]:
### Generate a dictionary with row as keys
indices = sparse_map[0].unique()
rowmap = {}
for i in indices:
    cols = sparse_map[sparse_map[0]==i]
    rowmap[i] = [(j, s) for j, s in zip(cols[1], cols[2])]


9.53 ms ± 54.6 µs per loop (mean ± std. dev. of 100 runs, 100 loops each)


In [31]:
# Check the length of each key
[(c, len(rowmap[c])) for c in rowmap.keys()]

[(1, 934),
 (2, 827),
 (3, 541),
 (4, 1316),
 (5, 567),
 (6, 1963),
 (7, 535),
 (8, 1742),
 (9, 765),
 (10, 724),
 (11, 86)]

In [94]:
# reconstruct whole matrix
# need to find a way of reconstructing with NaNs instead of 0s
rows, cols, vals = [], [], []
for key, values in rowmap.items():
    for value in values:
        rows.append(key)
        cols.append(value[0])
        vals.append(value[1])
X = sp.csr_matrix((vals, (rows, cols)))

In [92]:
# reconstruct certain rows
rows, cols, vals =[], [], []
row = [1,2,11] # list of row index
sub_dict = dict((k, rowmap[k]) for k in row if k in rowmap)
                
for key, values in sub_dict.items():
    for value in values:
        rows.append(key)
        cols.append(value[0])
        vals.append(value[1])
X = sp.csr_matrix((vals, (rows, cols)))

# Save the dictionary as json file and load it

In [99]:
import json

In [107]:
for key in rowmap.keys():
    if type(key) is not str:
        try:
            rowmap[str(key)] = rowmap[key]
        except:
            try:
                rowmap[repr(key)] = rowmap[key]
            except:
                pass
        del rowmap[key]

In [108]:
# save
with open('rowmap_dict.json', 'w') as fp:
    json.dump(rowmap, fp)

In [109]:
# load
with open('rowmap_dict.json', 'r') as fp:
    data = json.load(fp)